### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
! pip install implicit

     |████████████████████████████████| 1.1MB 2.6MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419511 sha256=1c9c12f82c427b95b2f8836816c94183095f752f21be79decf6ad3d271de0e6c
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [2]:
! pip install lightfm

     |████████████████████████████████| 307kB 2.6MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707631 sha256=c60328b3a132485e8f1ad4fba16af78fa0cc1d9ca38e080d2d92aecfc1810c5c
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [3]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
from typing import Union
from collections import Counter
from sklearn.neighbors import KDTree
import time
from lightfm.datasets import fetch_movielens
from itertools import product
from scipy.special import expit

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [5]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [6]:
movie_info = pd.read_csv('movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

In [7]:
movie_info

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


Explicit данные

In [8]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [9]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [10]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [11]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [12]:
print(user_item_csr.todense())

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [13]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [ ]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

In [ ]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [ ]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [ ]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

# Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

## Обработка explicit данных

In [15]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [16]:
ratings.rating.describe()

count    1.000209e+06
mean     3.581564e+00
std      1.117102e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [17]:
Counter(ratings.rating)

Counter({1: 56174, 2: 107557, 3: 261197, 4: 348971, 5: 226310})

В табличке есть все айдишники user-ов без пропусков (то есть, все числа от 1 до max --- айдишники).

In [18]:
all_users = np.unique(ratings["user_id"])
set(range(np.min(all_users), np.max(all_users) + 1)) - set(all_users)

set()

А вот каких-то фильмов в таблице нет.

In [19]:
all_movies = np.unique(ratings["movie_id"])
len(set(range(np.min(all_movies), np.max(all_movies) + 1)) - set(all_movies))

246

При этом множества айдишников в фильмах в двух таблицах совпадают.

In [20]:
len(set(np.unique(movie_info.movie_id)).intersection(set(all_movies))) == len(set(all_movies))

True

Заведем внутренние айдишники для пользователей и фильмов.

In [21]:
user2idx = dict(zip(all_users, range(len(all_users))))
idx2user = dict(zip(range(len(all_users)), all_users))

In [22]:
movie2idx = dict(zip(all_movies, range(len(all_movies))))
idx2movie = dict(zip(range(len(all_movies)), all_movies))

И заведем матрицу рейтинга во внутренних айдишниках.

In [23]:
users_idxs = [user2idx[user] for user in ratings["user_id"]]
movies_idxs = [movie2idx[movie] for movie in ratings["movie_id"]]
user_item = sp.coo_matrix((ratings["rating"], (users_idxs, movies_idxs)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [24]:
user_item_csr

<6040x3706 sparse matrix of type '<class 'numpy.longlong'>'
	with 1000209 stored elements in Compressed Sparse Row format>

## Реализация SVD

In [ ]:
class SVD():
    
    def __init__(self, 
                 n_users: int, 
                 n_items: int, 
                 latent_dim: int = 128, 
                 max_iter: int = 1e6, 
                 eps: float = 1., 
                 check_every = 100,
                 check_rmse_on = 1000,
                 lr: float = 1e-2, 
                 reg: dict = dict(zip(['u', 'v', 'b_u', 'b_v'], [1e-2, 1e-2, 1e-1, 1e-1])), 
                 seed: int = 73):
        self.n_users = n_users
        self.n_items = n_items
        self.max_iter = max_iter
        self.check_every = check_every
        self.check_rmse_on = check_rmse_on
        self.eps = eps
        self.latent_dim = latent_dim
        self.lr = lr
        self.reg = reg
        self.seed = seed
        np.random.seed(self.seed)
        self.U = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_users, latent_dim))
        self.V = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_items, latent_dim))
        self.mu = None
        self.b_u = None
        self.b_v = None
        #self.b_u = np.zeros(shape=(n_users, ), dtype='float32')
        #self.b_v = np.zeros(shape=(n_items, ), dtype='float32')
        
    
    def fit(self, X: sp.csr.csr_matrix):
        assert self.n_users, self.n_items == X.shape
        
        self.X = X
        self.mu = np.mean(X.data)
        self.b_u = np.array(X.mean(axis=1))
        self.b_v = np.array(X.mean(axis=0)).reshape(-1, )

        Q = 10000 + self.eps
        cur_iter = 0
        rand_indexes = np.random.randint(X.nnz, size=(self.max_iter, ))
        coords = X.nonzero()
        i_coords = coords[0][rand_indexes]
        j_coords = coords[1][rand_indexes]
        values = X.data[rand_indexes]
        while cur_iter < self.max_iter and Q > self.eps:
            
            # Select random nonzero x_ij
            i = i_coords[cur_iter]
            j = j_coords[cur_iter]
            x_ij = values[cur_iter]
            
            # Calculate gradients
            error = np.dot(self.U[i, :], self.V[j, :]) + self.mu + self.b_u[i] + self.b_v[j] - x_ij
            du_ij = error * self.V[j, :] + self.reg['u'] * self.U[i, :]
            dv_ij = error * self.U[i, :] + self.reg['v'] * self.V[j, :]
            dmu = error
            db_u_i = error + self.reg['b_u'] * self.b_u[i]
            db_v_j = error + self.reg['b_v'] * self.b_v[j]
            
            # Update
            self.U[i, :] -= self.lr * du_ij
            self.V[j, :] -= self.lr * dv_ij
            self.mu -= self.lr * dmu
            self.b_u[i] -= self.lr * db_u_i
            self.b_v[j] -= self.lr * db_v_j
             
            
            if not cur_iter % self.check_every:
                # We calculate RMSE on self.check_rmse_on random user-item pair 
                # (with known rating score) 
                rand_indexes2check = np.random.randint(X.nnz, size=(self.check_rmse_on, ))
                nonzero_inds = X.nonzero()
                nonzero_i = nonzero_inds[0][rand_indexes2check]
                nonzero_j = nonzero_inds[1][rand_indexes2check]
                nonzero_values = X.data[rand_indexes2check]
                Q = 0
                for value, i, j in zip(list(nonzero_values), list(nonzero_i), list(nonzero_j)):
                    Q += (self.U[i, :] @ self.V[j, :].T + self.mu + 
                         self.b_u[i] + self.b_v[j] - value) ** 2
                Q = np.sqrt(Q / self.check_rmse_on)
                print(f'iter: {cur_iter}, RMSE value: {Q}')
            cur_iter += 1
        self.U_KDTree = KDTree(self.U)
        self.V_KDTree = KDTree(self.V)
        
    
    def get_similars_items(self, item_ids: Union[int, list, np.ndarray], num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.V_KDTree.query(self.V[item_ids, :], k=num_neighbors, return_distance=False)   
        return self.V_KDTree.query([self.V[item_ids, :]], k=num_neighbors, return_distance=False)
    
    def get_similars_users(self, user_ids: Union[int, list, np.ndarray], num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.U_KDTree.query(self.U[user_ids, :], k=num_neighbors, return_distance=False)   
        return self.U_KDTree.query([self.U[item_ids, :]], k=num_neighbors, return_distance=False)
    
    def get_history(self, user_id: int):
        return [it_id for it_id in self.X.getrow(user_id).nonzero()[1]]

    def get_recommends(self, user_id: int, num_recs: int = 10):
        rating = self.X.getrow(user_id)
        
        not_known_items = np.array(list(
            set(range(self.X.shape[1])) - set(rating.nonzero()[1])))
        score = (self.U[user_id, :] @ self.V[not_known_items, :].T + 
                 self.mu + self.b_u[user_id] + self.b_v[not_known_items])
        order = np.argsort(score)
        order = order[::-1]
        return not_known_items[order[: num_recs]]

## Применение SVD

In [ ]:
%%time 
svd = SVD(n_users=user_item_csr.shape[0], n_items=user_item_csr.shape[1], 
          latent_dim=64, max_iter=int(5e6), check_every=int(1e5), check_rmse_on=2000, eps=0.)
svd.fit(user_item_csr)


iter: 0, RMSE value: [1.61153818]
iter: 100000, RMSE value: [0.98351851]
iter: 200000, RMSE value: [0.98396521]
iter: 300000, RMSE value: [0.95446969]
iter: 400000, RMSE value: [0.94177664]
iter: 500000, RMSE value: [0.90513]
iter: 600000, RMSE value: [0.92932844]
iter: 700000, RMSE value: [0.92962365]
iter: 800000, RMSE value: [0.9358738]
iter: 900000, RMSE value: [0.92402081]
iter: 1000000, RMSE value: [0.89610727]
iter: 1100000, RMSE value: [0.91280357]
iter: 1200000, RMSE value: [0.93196971]
iter: 1300000, RMSE value: [0.93113327]
iter: 1400000, RMSE value: [0.91578552]
iter: 1500000, RMSE value: [0.91284055]
iter: 1600000, RMSE value: [0.89900201]
iter: 1700000, RMSE value: [0.92119904]
iter: 1800000, RMSE value: [0.90358924]
iter: 1900000, RMSE value: [0.89962986]
iter: 2000000, RMSE value: [0.90119802]
iter: 2100000, RMSE value: [0.90539967]
iter: 2200000, RMSE value: [0.87164021]
iter: 2300000, RMSE value: [0.88890274]
iter: 2400000, RMSE value: [0.89633983]
iter: 2500000, RMSE

Посмотрим на фильмы, похожие на историю игрушек

In [ ]:
similar_movies = svd.get_similars_items(movie2idx[1])[0]
similar_movies = [idx2movie[idx] for idx in similar_movies]

In [ ]:
movie_info.set_index('movie_id').loc[similar_movies, :]

,name,category
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
3114,Toy Story 2 (1999),Animation|Children's|Comedy
588,Aladdin (1992),Animation|Children's|Comedy|Musical
1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
595,Beauty and the Beast (1991),Animation|Children's|Musical
953,It's a Wonderful Life (1946),Drama
2090,"Rescuers, The (1977)",Animation|Children's
364,"Lion King, The (1994)",Animation|Children's|Musical
2085,101 Dalmatians (1961),Animation|Children's


In [ ]:
hist = svd.get_history(user2idx[4])
hist = [idx2movie[movie] for movie in hist]
movie_info.set_index('movie_id').loc[hist, :]

,name,category
movie_id,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1036,Die Hard (1988),Action|Thriller
1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1198,Raiders of the Lost Ark (1981),Action|Adventure
1201,"Good, The Bad and The Ugly, The (1966)",Action|Western
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1214,Alien (1979),Action|Horror|Sci-Fi|Thriller


In [ ]:
recs = svd.get_recommends(user2idx[4], 10)
recs = [idx2movie[rec] for rec in recs]
movie_info.set_index('movie_id').loc[recs, :]

,name,category
movie_id,,
2905,Sanjuro (1962),Action|Adventure
858,"Godfather, The (1972)",Action|Crime|Drama
3022,"General, The (1927)",Comedy
527,Schindler's List (1993),Drama|War
1148,"Wrong Trousers, The (1993)",Animation|Comedy
912,Casablanca (1942),Drama|Romance|War
1178,Paths of Glory (1957),Drama|War
913,"Maltese Falcon, The (1941)",Film-Noir|Mystery
745,"Close Shave, A (1995)",Animation|Comedy|Thriller


# Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

Реализация статьи [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)

In [ ]:
class ALS:
    def __init__(self, 
                 n_users: int, 
                 n_items: int, 
                 latent_dim: int = 128, 
                 num_epochs: int = 1e6, 
                 eps: float = 0., 
                 check_rmse_on = 1000,
                 alpha: int = 40,
                 lr: float = 1e-2, 
                 reg: float = 1e-2,
                 seed: int = 73):
        
        self.n_users = n_users
        self.n_items = n_items
        self.latent_dim = latent_dim
        self.eps = eps
        self.check_rmse_on = check_rmse_on
        self.num_epochs = num_epochs
        self.reg = reg
        self.seed = seed
        self.alpha = alpha
        np.random.seed(seed)

        self.U = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_users, latent_dim))
        self.V = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_items, latent_dim))
    
    def check_rmse(self):
        #check RMSE
        # We calculate RMSE on self.check_rmse_on random user-item pair 
        # (with known rating score) 
        rand_indexes2check = np.random.randint(self.X.nnz, 
                                               size=(self.check_rmse_on, ))
        nonzero_inds = self.X.nonzero()
        nonzero_i = nonzero_inds[0][rand_indexes2check]
        nonzero_j = nonzero_inds[1][rand_indexes2check]
        nonzero_values = self.X.data[rand_indexes2check]
        Q = 0
        for value, i, j in zip(list(nonzero_values), list(nonzero_i), list(nonzero_j)):
            Q += (self.U[i, :] @ self.V[j, :].T - value) ** 2
        Q = np.sqrt(Q / self.check_rmse_on)
        print(f'RMSE value: {Q}')
        return Q
    
    def fit(self, X: sp.csr.csr_matrix):
        assert self.n_users, self.n_items == X.shape

        self.X = X
        Xt = X.transpose()
        cur_epoch = 0               
        Q = self.check_rmse()
            
        while cur_epoch < self.num_epochs and Q > self.eps:
            print(f'Epoch: {cur_epoch + 1}')

            # loop on users
            VtV = self.V.T @ self.V
            for i in range(self.n_users):
                if not i % 1000:
                    print(f'Users: [{i}/{self.n_users}]')
                r_u = X.getrow(i).toarray()[0]
                C_u = sp.diags(1 + self.alpha * r_u, offsets=0)           
                p_u = r_u > 0
                VtC_uV = (VtV + self.V.T.dot(sp.csr_matrix.dot(C_u - sp.identity(self.n_items), self.V)))
                inv = np.linalg.inv(VtC_uV + self.reg * sp.identity(VtC_uV.shape[0]))
                self.U[i, :] = inv @ self.V.T @ C_u @ p_u
            print('^_^')
            
            #loop on items
            UtU = self.U.T @ self.U
            for j in range(self.n_items):
                if not j % 1000:
                    print(f'Items: [{j}/{self.n_items}]')
                r_v = Xt.getrow(j).toarray()[0]
                C_v = sp.diags(1 + self.alpha * r_v, offsets=0)
                p_v = r_v > 0
                UtC_vU = (UtU + self.U.T.dot(sp.csr_matrix.dot(C_v - sp.identity(self.n_users), self.U)))
                inv = np.linalg.inv(UtC_vU + self.reg * sp.identity(UtC_vU.shape[0]))
                self.V[j, :] = inv @ self.U.T @ C_v @ p_v
            
            cur_epoch += 1
            Q = self.check_rmse()
        self.U_KDTree = KDTree(self.U)
        self.V_KDTree = KDTree(self.V)
        
    def get_similars_items(self, item_ids: Union[int, list, np.ndarray], num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.V_KDTree.query(self.V[item_ids, :], k=num_neighbors, return_distance=False)   
        return self.V_KDTree.query([self.V[item_ids, :]], k=num_neighbors, return_distance=False)
    
    def get_similars_users(self, user_ids: Union[int, list, np.ndarray], num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.U_KDTree.query(self.U[user_ids, :], k=num_neighbors, return_distance=False)   
        return self.U_KDTree.query([self.U[item_ids, :]], k=num_neighbors, return_distance=False)
    
    def get_history(self, user_id: int):
        return [it_id for it_id in self.X.getrow(user_id).nonzero()[1]]

    def get_recommends(self, user_id: int, num_recs: int = 10):
        rating = self.X.getrow(user_id)
        
        not_known_items = np.array(list(
            set(range(self.X.shape[1])) - set(rating.nonzero()[1])))
        score = self.U[user_id, :] @ self.V[not_known_items, :].T
        order = np.argsort(score)
        order = order[::-1]
        return not_known_items[order[: num_recs]]




## Подготовка данных

In [26]:
users_idxs = [user2idx[user] for user in ratings["user_id"]]
movies_idxs = [movie2idx[movie] for movie in ratings["movie_id"]]
user_item = sp.coo_matrix((np.ones_like(users_idxs), (users_idxs, movies_idxs)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

## Применение ALS

In [ ]:
als = ALS(n_users=user_item.shape[0], n_items=user_item.shape[1], num_epochs=20, eps=1e-1, latent_dim=64, alpha=20)

In [ ]:
als.fit(user_item_csr)

RMSE value: 0.750188829785412
Epoch: 1
Users: [0/6040]
Users: [1000/6040]
Users: [2000/6040]
Users: [3000/6040]
Users: [4000/6040]
Users: [5000/6040]
Users: [6000/6040]
^_^
Items: [0/3706]
Items: [1000/3706]
Items: [2000/3706]
Items: [3000/3706]
RMSE value: 0.28070557195825874
Epoch: 2
Users: [0/6040]
Users: [1000/6040]
Users: [2000/6040]
Users: [3000/6040]
Users: [4000/6040]
Users: [5000/6040]
Users: [6000/6040]
^_^
Items: [0/3706]
Items: [1000/3706]
Items: [2000/3706]
Items: [3000/3706]
RMSE value: 0.22548378600838517
Epoch: 3
Users: [0/6040]
Users: [1000/6040]
Users: [2000/6040]
Users: [3000/6040]
Users: [4000/6040]
Users: [5000/6040]
Users: [6000/6040]
^_^
Items: [0/3706]
Items: [1000/3706]
Items: [2000/3706]
Items: [3000/3706]
RMSE value: 0.21603888652460043
Epoch: 4
Users: [0/6040]
Users: [1000/6040]
Users: [2000/6040]
Users: [3000/6040]
Users: [4000/6040]
Users: [5000/6040]
Users: [6000/6040]
^_^
Items: [0/3706]
Items: [1000/3706]
Items: [2000/3706]
Items: [3000/3706]
RMSE value

In [ ]:
similar_movies = als.get_similars_items(movie2idx[1])[0]
similar_movies = [idx2movie[idx] for idx in similar_movies]
movie_info.set_index('movie_id').loc[similar_movies, :]

,name,category
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
3114,Toy Story 2 (1999),Animation|Children's|Comedy
34,Babe (1995),Children's|Comedy|Drama
1265,Groundhog Day (1993),Comedy|Romance
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
2396,Shakespeare in Love (1998),Comedy|Romance
1923,There's Something About Mary (1998),Comedy
2321,Pleasantville (1998),Comedy
1641,"Full Monty, The (1997)",Comedy


In [ ]:
hist = als.get_history(user2idx[4])
hist = [idx2movie[movie] for movie in hist]
movie_info.set_index('movie_id').loc[hist, :]

,name,category
movie_id,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1036,Die Hard (1988),Action|Thriller
1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1198,Raiders of the Lost Ark (1981),Action|Adventure
1201,"Good, The Bad and The Ugly, The (1966)",Action|Western
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1214,Alien (1979),Action|Horror|Sci-Fi|Thriller


In [ ]:
recs = als.get_recommends(user2idx[4], 10)
recs = [idx2movie[rec] for rec in recs]
movie_info.set_index('movie_id').loc[recs, :]

,name,category
movie_id,,
1200,Aliens (1986),Action|Sci-Fi|Thriller|War
858,"Godfather, The (1972)",Action|Crime|Drama
1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
457,"Fugitive, The (1993)",Action|Thriller
1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
110,Braveheart (1995),Action|Drama|War
3703,Mad Max 2 (a.k.a. The Road Warrior) (1981),Action|Sci-Fi


# Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

Здесь мы максимизируем правдоподобие, поэтому в качестве лосса вывожу -log_likelihood / num_triplets (нормирую, чтобы было удобно останавливаться).

In [153]:
class BPR:
    def __init__(self, 
                 n_users: int, 
                 n_items: int, 
                 latent_dim: int = 128, 
                 num_epochs: int = 100, 
                 neg_size: int = 5,
                 eps: float = 0., 
                 check_loss_on = 1000,
                 lr: float = 1e-2, 
                 reg: float = 1e-2,
                 seed: int = 73):
        
        self.n_users = n_users
        self.n_items = n_items
        self.latent_dim = latent_dim
        self.neg_size = neg_size
        self.eps = eps
        self.check_loss_on = check_loss_on
        self.num_epochs = num_epochs
        self.reg = reg
        self.seed = seed
        self.lr = lr
        np.random.seed(seed)

        self.U = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_users, latent_dim))
        self.V = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_items, latent_dim))
    
    def check_loss(self):
        #check likelihood
        # We calculate likelihood on self.check_loss_on random user-item-neg_item triplets
        rand_indexes2check = np.random.randint(self.X.nnz, 
                                               size=(self.check_loss_on, ))
        nonzero_inds = self.X.nonzero()
        nonzero_u = nonzero_inds[0][rand_indexes2check]
        nonzero_i = nonzero_inds[1][rand_indexes2check]
        L = 0
        for user, item in zip(list(nonzero_u), list(nonzero_i)):
            negative = np.random.choice(self.negative_items[user], 
                                        self.neg_size, replace=False)
            x_uij = (self.V[item, :] - self.V[negative, :]) @ self.U[user, :]
            L += -np.log(1 + np.exp(-x_uij)).sum(axis=0)
        L /= self.check_loss_on * self.neg_size
        L = -L
        print(f'Loss value: {L}')
        return L


    def fit(self, X: sp.csr.csr_matrix):
        assert self.n_users, self.n_items == X.shape

        self.X = X
        cur_epoch = 0               
        
        self.positive_items = {}
        self.negative_items = {}

        for user in range(self.n_users):
            rating = self.X.getrow(user)
            self.positive_items[user] = rating.nonzero()[1]
            self.negative_items[user] = np.array(list(
                set(range(self.X.shape[1])) - set(self.positive_items[user])))
        L = self.check_loss()

        
        while cur_epoch < self.num_epochs and L > self.eps:
            print(f'Epoch: {cur_epoch + 1}')

            for i, user in enumerate(np.random.permutation(range(self.n_users))):
                for item in np.random.permutation(self.positive_items[user]):

                    negative = np.random.choice(self.negative_items[user], 
                                                self.neg_size, replace=False)
                    
                    diff_V = self.V[item, :] - self.V[negative, :]
                    x_uij = diff_V @ self.U[user, :]
                    coeff = 1 / (1 + np.exp(x_uij))
                    coeff_mult_U = np.outer(coeff, self.U[user, :])

                    self.V[negative, :] += self.lr * (-coeff_mult_U - 
                                                      self.reg * self.V[negative, :])
                    self.V[item, :] += self.lr * (coeff_mult_U.sum(axis=0) - 
                                                  self.reg * self.V[item, :])
                    self.U[user, :] += self.lr * ((
                        coeff.reshape((-1, 1)) * diff_V).sum(axis=0) - 
                                                  self.reg * self.U[user, :])
                                 
                if not i % 1000:
                    self.check_loss()
           
            cur_epoch += 1
            L = self.check_loss()
    
        self.U_KDTree = KDTree(self.U)
        self.V_KDTree = KDTree(self.V)
        
    def get_similars_items(self, item_ids: Union[int, list, np.ndarray], 
                           num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.V_KDTree.query(self.V[item_ids, :], 
                                       k=num_neighbors, return_distance=False)   
        return self.V_KDTree.query([self.V[item_ids, :]], 
                                   k=num_neighbors, return_distance=False)
    
    def get_similars_users(self, user_ids: Union[int, list, np.ndarray], 
                           num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.U_KDTree.query(self.U[user_ids, :], k=num_neighbors, 
                                       return_distance=False)   
        return self.U_KDTree.query([self.U[item_ids, :]], k=num_neighbors, 
                                   return_distance=False)
    
    def get_history(self, user_id: int):
        return [it_id for it_id in self.X.getrow(user_id).nonzero()[1]]

    def get_recommends(self, user_id: int, num_recs: int = 10):
        rating = self.X.getrow(user_id)
        
        not_known_items = np.array(list(
            set(range(self.X.shape[1])) - set(rating.nonzero()[1])))
        score = self.U[user_id, :] @ self.V[not_known_items, :].T
        order = np.argsort(score)
        order = order[::-1]
        return not_known_items[order[: num_recs]]




## Запуск BPR

In [154]:
bpr = BPR(n_users=user_item.shape[0], n_items=user_item.shape[1], num_epochs=5, eps=1e-1, latent_dim=128, neg_size=10, lr=1e-2, reg=1e-4)

In [155]:
bpr.fit(user_item_csr)

Loss value: 0.693249064778535
Epoch: 1
Loss value: 0.6920902098558579
Loss value: 0.448211100027927
Loss value: 0.4118927335788052
Loss value: 0.3657205202283755
Loss value: 0.34147443072825323
Loss value: 0.3001819688521727
Loss value: 0.25728320165886515
Loss value: 0.24429711760442202
Epoch: 2
Loss value: 0.24908759895614352
Loss value: 0.22331711957460443
Loss value: 0.22043213124722882
Loss value: 0.21999396684746025
Loss value: 0.22603370598412492
Loss value: 0.20069606019390154
Loss value: 0.18194131140061143
Loss value: 0.17313953149418168
Epoch: 3
Loss value: 0.16982891340465395
Loss value: 0.17242022258944356
Loss value: 0.19849515397983974
Loss value: 0.1745308057790078
Loss value: 0.17073378400503977
Loss value: 0.16343709455730732
Loss value: 0.15455988455549552
Loss value: 0.15355115792201912
Epoch: 4
Loss value: 0.1653348178006134
Loss value: 0.16021369626157475
Loss value: 0.1500608087868829
Loss value: 0.14592689645545337
Loss value: 0.14003361928743635
Loss value: 0.1

In [156]:
similar_movies = bpr.get_similars_items(movie2idx[1])[0]
similar_movies = [idx2movie[idx] for idx in similar_movies]
movie_info.set_index('movie_id').loc[similar_movies, :]

,name,category
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
34,Babe (1995),Children's|Comedy|Drama
3114,Toy Story 2 (1999),Animation|Children's|Comedy
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
588,Aladdin (1992),Animation|Children's|Comedy|Musical
1265,Groundhog Day (1993),Comedy|Romance
2384,Babe: Pig in the City (1998),Children's|Comedy
2321,Pleasantville (1998),Comedy
356,Forrest Gump (1994),Comedy|Romance|War


In [157]:
hist = bpr.get_history(user2idx[4])
hist = [idx2movie[movie] for movie in hist]
movie_info.set_index('movie_id').loc[hist, :]

,name,category
movie_id,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1036,Die Hard (1988),Action|Thriller
1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1198,Raiders of the Lost Ark (1981),Action|Adventure
1201,"Good, The Bad and The Ugly, The (1966)",Action|Western
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1214,Alien (1979),Action|Horror|Sci-Fi|Thriller


In [158]:
recs = bpr.get_recommends(user2idx[4], 10)
recs = [idx2movie[rec] for rec in recs]
movie_info.set_index('movie_id').loc[recs, :]

,name,category
movie_id,,
858,"Godfather, The (1972)",Action|Crime|Drama
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1270,Back to the Future (1985),Comedy|Sci-Fi
2858,American Beauty (1999),Comedy|Drama
541,Blade Runner (1982),Film-Noir|Sci-Fi
110,Braveheart (1995),Action|Drama|War
608,Fargo (1996),Crime|Drama|Thriller
593,"Silence of the Lambs, The (1991)",Drama|Thriller


# Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [145]:
class WARP:
    def __init__(self, 
                 n_users: int, 
                 n_items: int, 
                 latent_dim: int = 128, 
                 num_epochs: int = 100, 
                 neg_size: int = 5,
                 eps: float = 0., 
                 check_loss_on = 1000,
                 lr: float = 1e-2, 
                 reg: float = 1e-2,
                 seed: int = 73):
        
        self.n_users = n_users
        self.n_items = n_items
        self.latent_dim = latent_dim
        self.neg_size = neg_size
        self.eps = eps
        self.check_loss_on = check_loss_on
        self.num_epochs = num_epochs
        self.reg = reg
        self.seed = seed
        self.lr = lr
        np.random.seed(seed)

        self.U = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_users, latent_dim))
        self.V = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_items, latent_dim))
    
    def check_loss(self):
        #check loss
        # We calculate loss on self.check_loss_on random user-item-neg_item triplets
        rand_indexes2check = np.random.randint(self.X.nnz, 
                                               size=(self.check_loss_on, ))
        nonzero_inds = self.X.nonzero()
        nonzero_u = nonzero_inds[0][rand_indexes2check]
        nonzero_i = nonzero_inds[1][rand_indexes2check]
        L = 0
        for user, item in zip(list(nonzero_u), list(nonzero_i)):
            negative = np.random.choice(self.negative_items[user], 
                                        self.neg_size, replace=False)
            for q, neg in enumerate(negative):
                diff_V = self.V[item, :] - self.V[neg, :]
                diff_pos_neg = self.U[user, :] @ diff_V

                if 1 - diff_pos_neg <= 0:
                    continue
                    
                L += np.log(len(self.negative_items[user]) / (q + 1))
                break
        L /= self.check_loss_on
        #L = -L
        print(f'Loss value: {L}')
        return L


    def fit(self, X: sp.csr.csr_matrix):
        assert self.n_users, self.n_items == X.shape

        self.X = X
        cur_epoch = 0               
        
        self.positive_items = {}
        self.negative_items = {}

        for user in range(self.n_users):
            rating = self.X.getrow(user)
            self.positive_items[user] = rating.nonzero()[1]
            self.negative_items[user] = np.array(list(
                set(range(self.X.shape[1])) - set(self.positive_items[user])))
        
        L = self.check_loss()

        
        while cur_epoch < self.num_epochs and L > self.eps:
            print(f'Epoch: {cur_epoch + 1}')

            for i, user in enumerate(np.random.permutation(range(self.n_users))):
                for item in np.random.permutation(self.positive_items[user]):

                    negative = np.random.choice(self.negative_items[user], 
                                                self.neg_size, replace=False)

                    for q, neg in enumerate(negative):

                        diff_V = self.V[item, :] - self.V[neg, :]
                        diff_pos_neg = self.U[user, :] @ diff_V

                        if 1 - diff_pos_neg <= 0:
                            continue

                        coeff = np.log(len(self.negative_items[user]) / (q + 1))
                        
                        coeff_mult_U = coeff * self.U[user, :]

                        self.V[neg, :] += self.lr * (-coeff_mult_U - 
                                                     self.reg * self.V[neg, :])
                        self.V[item, :] += self.lr * (coeff_mult_U - 
                                                      self.reg * self.V[item, :])
                        self.U[user, :] += self.lr * (coeff * diff_V - 
                                                      self.reg * self.U[user, :])
                        break
                                 
                if not i % 1000:
                    self.check_loss()
           
            cur_epoch += 1
            L = self.check_loss()
    
        self.U_KDTree = KDTree(self.U)
        self.V_KDTree = KDTree(self.V)
        
    def get_similars_items(self, item_ids: Union[int, list, np.ndarray], 
                           num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.V_KDTree.query(self.V[item_ids, :], 
                                       k=num_neighbors, return_distance=False)   
        return self.V_KDTree.query([self.V[item_ids, :]], 
                                   k=num_neighbors, return_distance=False)
    
    def get_similars_users(self, user_ids: Union[int, list, np.ndarray], 
                           num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.U_KDTree.query(self.U[user_ids, :], k=num_neighbors, 
                                       return_distance=False)   
        return self.U_KDTree.query([self.U[item_ids, :]], k=num_neighbors, 
                                   return_distance=False)
    
    def get_history(self, user_id: int):
        return [it_id for it_id in self.X.getrow(user_id).nonzero()[1]]

    def get_recommends(self, user_id: int, num_recs: int = 10):
        rating = self.X.getrow(user_id)
        
        not_known_items = np.array(list(
            set(range(self.X.shape[1])) - set(rating.nonzero()[1])))
        score = self.U[user_id, :] @ self.V[not_known_items, :].T
        order = np.argsort(score)
        order = order[::-1]
        return not_known_items[order[: num_recs]]




In [164]:
warp = WARP(n_users=user_item.shape[0], n_items=user_item.shape[1], num_epochs=5, eps=1e-1, latent_dim=64, neg_size=20, lr=1e-2, reg=1e-3)

In [165]:
warp.fit(user_item_csr)

Loss value: 8.095505238567386
Epoch: 1
Loss value: 8.102160605771589
Loss value: 7.152108958920731
Loss value: 6.3536152788026685
Loss value: 5.560800937765715
Loss value: 5.191724226324982
Loss value: 4.907976092066809
Loss value: 4.073299652931411
Loss value: 4.090535153607938
Epoch: 2
Loss value: 4.085043574589238
Loss value: 4.056012614192793
Loss value: 3.9775527158429194
Loss value: 3.7710103287815695
Loss value: 3.9277136705363467
Loss value: 3.7680880194394315
Loss value: 3.651630647178534
Loss value: 3.4781327688524226
Epoch: 3
Loss value: 3.6550861940503836
Loss value: 3.5602648885637005
Loss value: 3.3910673918248238
Loss value: 3.523824146950036
Loss value: 3.3892277621962004
Loss value: 3.4060740517757755
Loss value: 3.3510383182675834
Loss value: 3.313588853232597
Epoch: 4
Loss value: 3.1461513333871376
Loss value: 3.3926666277720616
Loss value: 3.3740971457784155
Loss value: 3.2711635716444496
Loss value: 3.0589229883751643
Loss value: 3.2420024338411673
Loss value: 3.32

In [169]:
similar_movies = warp.get_similars_items(movie2idx[1])[0]
similar_movies = [idx2movie[idx] for idx in similar_movies]
movie_info.set_index('movie_id').loc[similar_movies, :]

,name,category
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
3114,Toy Story 2 (1999),Animation|Children's|Comedy
34,Babe (1995),Children's|Comedy|Drama
296,Pulp Fiction (1994),Crime|Drama
2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
1394,Raising Arizona (1987),Comedy
588,Aladdin (1992),Animation|Children's|Comedy|Musical
1641,"Full Monty, The (1997)",Comedy
1265,Groundhog Day (1993),Comedy|Romance


In [170]:
hist = warp.get_history(user2idx[4])
hist = [idx2movie[movie] for movie in hist]
movie_info.set_index('movie_id').loc[hist, :]

,name,category
movie_id,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1036,Die Hard (1988),Action|Thriller
1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1198,Raiders of the Lost Ark (1981),Action|Adventure
1201,"Good, The Bad and The Ugly, The (1966)",Action|Western
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1214,Alien (1979),Action|Horror|Sci-Fi|Thriller


In [171]:
recs = warp.get_recommends(user2idx[4], 10)
recs = [idx2movie[rec] for rec in recs]
movie_info.set_index('movie_id').loc[recs, :]

,name,category
movie_id,,
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1200,Aliens (1986),Action|Sci-Fi|Thriller|War
1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2916,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller
1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
110,Braveheart (1995),Action|Drama|War
858,"Godfather, The (1972)",Action|Crime|Drama
2858,American Beauty (1999),Comedy|Drama
